In [4]:
import numpy as np
import random
import time
import tensorflow as tf
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [5]:
GRAPH = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 4],
    3: [0, 4],
    4: [1, 2, 3]
}
N_STATES = len(GRAPH)
N_ACTIONS = N_STATES
START_STATE = 0
GOAL_STATE = 4
POSITIVE_REWARD = 100
NEGATIVE_REWARD = -1

In [6]:
def get_reward(current_state, next_state):
    if next_state == GOAL_STATE:
        return POSITIVE_REWARD
    if next_state not in GRAPH.get(current_state, []):
        return NEGATIVE_REWARD * 10
    return NEGATIVE_REWARD

In [7]:
# --- 1. Tabular Reinforcement Learning (Q-Learning) ---

class QLearningAgent:
    def __init__(self, states, actions, lr=0.1, gamma=0.9, epsilon=1.0, min_epsilon=0.01, decay_rate=0.9995):
        self.states = states
        self.actions = actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.min_epsilon = min_epsilon
        self.decay_rate = decay_rate
        self.q_table = np.zeros((states, actions))

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.choice(GRAPH[state])
        else:
            q_values = self.q_table[state]
            valid_actions = GRAPH[state]
            best_action = valid_actions[np.argmax(q_values[valid_actions])]
            return best_action

    def update_q_table(self, state, action, reward, next_state):
        action_index = action
        valid_next_actions = GRAPH.get(next_state, [])
        if next_state == GOAL_STATE or not valid_next_actions:
            max_next_q = 0
        else:
            max_next_q = np.max(self.q_table[next_state][valid_next_actions])

        old_value = self.q_table[state, action_index]
        new_value = (1 - self.lr) * old_value + self.lr * (reward + self.gamma * max_next_q)
        self.q_table[state, action_index] = new_value

    def decay_epsilon(self):
        self.epsilon = max(self.min_epsilon, self.epsilon * self.decay_rate)


In [8]:
def run_q_learning(episodes=1000):
    agent = QLearningAgent(N_STATES, N_ACTIONS)
    total_steps = 0

    start_time = time.time()
    for episode in range(episodes):
        state = START_STATE
        step_count = 0
        done = False

        while not done:
            action = agent.choose_action(state)
            next_state = action
            reward = get_reward(state, next_state)

            agent.update_q_table(state, action, reward, next_state)

            state = next_state
            total_steps += 1
            step_count += 1

            if state == GOAL_STATE or step_count > N_STATES * 5:
                done = True

        agent.decay_epsilon()

    end_time = time.time()
    return end_time - start_time, total_steps, agent.q_table

This notebook implements a basic Q-Learning agent to find the optimal path in a predefined graph. It covers the following key aspects:

Environment Setup: Defines a graph (Markov Decision Process) with states and possible transitions, along with start and goal states and associated rewards.
Reward Function: A get_reward function calculates rewards based on state transitions, including positive rewards for reaching the goal and negative rewards for invalid or non-goal moves.
Q-Learning Agent: The QLearningAgent class implements the core Q-learning algorithm. It includes:
Initialization: Sets up the Q-table, learning rate (lr), discount factor (gamma), and exploration-exploitation parameters (epsilon).
Action Selection: Uses an epsilon-greedy policy to choose actions, balancing exploration (random actions) and exploitation (choosing actions with highest Q-value).
Q-table Update: Implements the Q-learning update rule to iteratively improve action-value estimates.
Epsilon Decay: Gradually reduces epsilon over time to shift from exploration to exploitation.
Training Loop: The run_q_learning function orchestrates the training process for a specified number of episodes, simulating interactions with the environment, updating the Q-table, and tracking performance metrics like training time and total steps. It also handles episode termination conditions (reaching goal or max steps).
The notebook provides a foundational example of reinforcement learning using Q-Learning for pathfinding in a simple discrete environment.